In [1]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [2]:
train.isnull().sum().sort_values(ascending = False)

Cabin          687
Age            177
Embarked         2
Fare             0
Ticket           0
Parch            0
SibSp            0
Sex              0
Name             0
Pclass           0
Survived         0
PassengerId      0
dtype: int64

In [3]:
test.isnull().sum().sort_values(ascending = False)

Cabin          327
Age             86
Fare             1
Embarked         0
Ticket           0
Parch            0
SibSp            0
Sex              0
Name             0
Pclass           0
PassengerId      0
dtype: int64

In [4]:
for i in train.columns:
    print(i, ':', train[i].nunique())

PassengerId : 891
Survived : 2
Pclass : 3
Name : 891
Sex : 2
Age : 88
SibSp : 7
Parch : 7
Ticket : 681
Fare : 248
Cabin : 147
Embarked : 3


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Survived'] = le.fit_transform(train['Survived'].astype('str'))

In [6]:
test['Survived'] = -1
df = pd.concat([train, test])

In [7]:
for i in ["Pclass", "Sex", "Embarked"]:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i].astype(str))

In [8]:
df['Age'].fillna(df['Age'].mode()[0], inplace = True)
df['Fare'].fillna(df['Fare'].mode()[0], inplace = True)

In [9]:
train = df[df['Survived']!=-1]
test = df[df['Survived']==-1]

In [10]:
train = train.drop(["Cabin", "Name", "Ticket"], axis = 1)
test = test.drop(["Cabin", "Survived", "Name", "Ticket"], axis = 1)

In [11]:
train.shape

(891, 9)

In [12]:
X1 = train.drop('Survived', axis =1)
y1 = train['Survived']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size =0.20, random_state =100)

In [13]:
import xgboost
classifier_xgb = xgboost.XGBClassifier(max_depth=4, learning_rate=0.01, n_estimators=2000,
                                  objective='binary:logistic', reg_alpha=0.5, reg_lambda=1.5,
                                  booster='gbtree', n_jobs=4, min_child_weight=2, base_score= 0.75)

In [14]:
model_xgb = classifier_xgb.fit(X_train, y_train)

In [15]:
prediction_xgb = model_xgb.predict(X_test)
from sklearn.metrics import accuracy_score
acc_score_xgb = accuracy_score(prediction_xgb,y_test)
print("Accuracy:", acc_score_xgb*100)

Accuracy: 83.79888268156425


In [16]:
pred_xgb = classifier_xgb.predict(test.iloc[:,:])
result_xgb = pd.DataFrame(pred_xgb, columns=['Survived'])
result_xgb['PassengerId'] = test['PassengerId']
result_xgb = result_xgb[['PassengerId', 'Survived']]

In [17]:
result_xgb.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [19]:
result_xgb.to_csv('Submission_xgb.csv', index=False, header = True)